In [ ]:
import numpy

In [ ]:
def bracket_term_ppe(b, rho, dt, u, v, dx, dy):
    b[1:-1, 1:-1] = (
        rho * (1 / dt * ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx) +
                         (v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy)) -
                        ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx))**2 -
                    2 * ((u[2:, 1:-1] - u[0:-2, 1:-1]) / (2 * dy) *
                         (v[1:-1, 2:] - v[1:-1, 0:-2]) / (2 * dx)) -
                        ((v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy))**2)
                    )

    return b


In [ ]:
%load_ext cython

In [ ]:
%%cython
cimport numpy
cimport cython

import numpy

from libc.math cimport sqrt

@cython.boundscheck(False)
def pressure_poisson(numpy.ndarray[numpy.float_t, ndim=2] p,
                     double dx, 
                     double dy, 
                     numpy.ndarray[numpy.float_t, ndim=2] b,
                     numpy.ndarray[numpy.float_t, ndim=2] pn,
                     double l2_target):
    
    cdef int j, i, n
    cdef double iter_diff
    cdef int J = b.shape[0]
    cdef int I = b.shape[1]    
    
    iter_diff = l2_target + 1
    
    while iter_diff > l2_target:
        n = 0
        pn = p.copy()
        for j in range(1, J - 1):
            for i in range(1, I - 1):
                p[j, i] = (((pn[j, i + 1] + pn[j, i - 1]) * dy**2 + 
                            (pn[j + 1, i] + pn[j - 1, i]) * dx**2) /
                            (2 * (dx**2 + dy**2)) -
                            dx**2 * dy**2 / (2 * (dx**2 + dy**2)) *
                            b[j, i])

        for j in range(J):
            p[j, 0] = p[j, 1]
            p[j, -1] = p[j, -2]
            
        for i in range(I):
            p[0, i] = p[1, i]
            p[-1, i] = 0
            
        
        if n % 10 == 0:
            iter_diff = L2_Cython(p, pn)
        if n == 500:
            break
            
        n += 1
        
    return p

@cython.boundscheck(False)
def L2_Cython(numpy.ndarray[numpy.float_t, ndim=2] p, 
              numpy.ndarray[numpy.float_t, ndim=2] pn):
    cdef double error = 0
    cdef double pnsum = 0
    cdef int ny = p.shape[0]
    cdef int nx = p.shape[1]
    for i in range(nx):
        for j in range(ny):
            error += (p[i,j]-pn[i,j])**2
            pnsum += pn[i,j]**2
            
    return sqrt(error/pnsum)

In [ ]:
def cavity_flow(nt, u, v, dt, dx, dy, p, rho, nu):
    un = numpy.empty_like(u)
    vn = numpy.empty_like(v)
    ny, nx = u.shape
    b = numpy.zeros((ny, nx))

    for n in range(nt):
        un = u.copy()
        vn = v.copy()

        b = bracket_term_ppe(b, rho, dt, u, v, dx, dy)
        p = pressure_poisson(p, dx, dy, b, numpy.empty_like(p), 1e-3)

        u[1:-1,1:-1] = (un[1:-1, 1:-1] -
                        un[1:-1,1:-1] * dt / dx *
                        (un[1:-1, 1:-1] - un[1:-1, 0:-2]) -
                        vn[1:-1, 1:-1 ] * dt / dy *
                        (un[1:-1, 1:-1] - un[0:-2, 1:-1]) -
                        dt / (2 * rho * dx) *
                        (p[1:-1, 2:] - p[1:-1, 0:-2]) +
                        nu * dt *
                        (1 / dx**2 *
                         (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2]) +
                        1 / dy**2 *
                         (un[2:, 1:-1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1])))

        v[1:-1,1:-1] = (vn[1:-1, 1:-1] -
                        un[1:-1, 1:-1] * dt / dx *
                        (vn[1:-1, 1:-1] - vn[1:-1, 0:-2]) -
                        vn[1:-1, 1:-1] * dt / dy *
                        (vn[1:-1, 1:-1] - vn[0:-2, 1:-1]) -
                        dt / (2 * rho * dy) *
                        (p[2:, 1:-1] - p[0:-2, 1:-1]) +
                        nu * dt *
                        (1 / dx**2 *
                         (vn[1:-1, 2:] - 2 * vn[1:-1, 1:-1] + vn[1:-1, 0:-2]) +
                        1 / dy**2 *
                         (vn[2:, 1:-1] - 2 * vn[1:-1, 1:-1] +vn[0:-2, 1:-1])))

        #remember order of these will have a (very small) effect on the 
        #final answers.  if you want to compare between versions, make sure
        #they match up
        u[:, 0] = 0
        u[:, -1] = 0
        v[:, 0] = 0
        v[:, -1] = 0
        u[0, :] = 0
        u[-1, :] = 1    #set velocity on cavity lid equal to 1
        v[0, :] = 0
        v[-1, :]=0
        
        
        
    return u, v, p

In [ ]:
import pickle

In [ ]:
def run_cavity():
    nx = 41
    ny = 41
    with open('IC.pickle', 'rb') as f:
        u, v, p, b = pickle.load(f)

    dx = 2 / (nx - 1)
    dy = 2 / (ny - 1)
    rho = 1
    nu = 0.1
    dt = .005

    nt = 500
    u, v, p = cavity_flow(nt, u, v, dt, dx, dy, p, rho, nu)
    
    return u, v, p

In [ ]:
import time

In [ ]:
tic = time.time()
u, v, p = run_cavity()
toc = time.time()

In [ ]:
toc-tic

In [ ]:
from matplotlib import pyplot, cm
%matplotlib inline

In [ ]:
nx = 41
ny = 41
x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)
X, Y = numpy.meshgrid(x, y)

In [ ]:
quiver_skip = qs = 4
pyplot.figure(figsize=(11, 7), dpi=100)
pyplot.contourf(X, Y, p, alpha=0.5, cmap=cm.viridis)
pyplot.colorbar()
pyplot.contour(X, Y, p)
pyplot.quiver(X[::qs, ::qs], Y[::qs, ::qs], u[::qs, ::qs], v[::qs, ::qs])

In [ ]:
with open('numpy_ans.pickle', 'rb') as f:
    unum, vnum, pnum = pickle.load(f)

In [ ]:
numpy.allclose(u, unum)

In [ ]:
numpy.allclose(v, vnum)

In [ ]:
numpy.allclose(p, pnum)

In [ ]:
%%timeit
run_cavity()

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f cavity_flow run_cavity()